In [1]:
from hw1 import Composer
from midi2seq import process_midi_seq, seq2piano, random_piano, piano2seq, segment
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [3]:
expert_seq = process_midi_seq(maxlen=50,n=12000)
expert_seq.shape

(13956, 51)

In [4]:
fake_midix = [random_piano() for i in range(1500)]
fake_seq = process_midi_seq(all_midis=fake_midix,maxlen=50,n=13000)
fake_seq.shape

(13019, 51)

In [5]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(-1,1))
# expert_seq = scaler.fit_transform(expert_seq)
# fake_seq = scaler.fit_transform(fake_seq)
# expert_seq.shape, fake_seq.shape

In [6]:
critic_data = np.zeros((expert_seq.shape[0] + fake_seq.shape[0], expert_seq.shape[1]+1))
critic_data[:expert_seq.shape[0],expert_seq.shape[1]] = 1
critic_data[:expert_seq.shape[0],:expert_seq.shape[1]] = expert_seq
critic_data[expert_seq.shape[0]:,:expert_seq.shape[1]] = fake_seq
critic_data, critic_data.shape

(array([[257., 355., 260., ...,  56., 287.,   1.],
        [257., 256., 363., ..., 262., 370.,   1.],
        [256., 184., 362., ..., 259., 256.,   1.],
        ...,
        [256., 358., 110., ...,  48., 294.,   0.],
        [238., 258., 256., ..., 174., 280.,   0.],
        [294., 256., 364., ..., 211., 265.,   0.]]),
 (26975, 52))

In [7]:
from sklearn.model_selection import train_test_split
train_sequences, test_sequences = train_test_split(critic_data , test_size=0.2)

In [8]:
train_sequences[:,51].tolist().count(0),  train_sequences[:,51].tolist().count(1)

(10374, 11206)

In [9]:
X_train = train_sequences[:,:51]
X_train = X_train.reshape((-1,51,1))

Y_train = train_sequences[:,51]
Y_train = Y_train.reshape((-1,1))

X_test = test_sequences[:,:51]
X_test = X_test.reshape((-1,51,1))

Y_test = test_sequences[:,51]
Y_test = Y_test.reshape((-1,1))



X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

X_test = torch.tensor(X_test).float()
Y_test = torch.tensor(Y_test).float()

X_train.shape, X_test.shape,  Y_train.shape, Y_test.shape

(torch.Size([21580, 51, 1]),
 torch.Size([5395, 51, 1]),
 torch.Size([21580, 1]),
 torch.Size([5395, 1]))

In [51]:
from torch.utils.data import Dataset
class MidiCriticDataset(Dataset):
    def __init__(self, X_sequence, Y_critic):
        self.X_sequence = X_sequence
        self.Y_critic = Y_critic

    def __len__(self):
        return len(self.Y_critic)
        
    def __getitem__(self, idx):
        sequence, label =  self.X_sequence[idx] ,self.Y_critic[idx]
        label = np.array([1, 0], dtype = float) if label else np.array([0, 1], dtype = float)
        return dict(
            sequence = sequence,
            label = label
        )

In [11]:
train_dataset = MidiCriticDataset(X_train,Y_train)
test_dataset = MidiCriticDataset(X_test,Y_test)

In [12]:
from torch.utils.data import DataLoader

BATCH_SIZE = 100

train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size = BATCH_SIZE, shuffle=False)

In [13]:
for _, batch in enumerate(train_loader):
    sequence_batch , label_batch = batch['sequence'].to(device) , batch['label'].to(device) 
    print(sequence_batch.shape, label_batch.shape)
    break

torch.Size([100, 51, 1]) torch.Size([100, 2])


In [14]:
class SequenceModel(nn.Module):
    def __init__(self, n_features, n_classes, n_hidden=256, n_layers=3):
        super().__init__()
        self.num_stacked_layers = n_layers
        self.hidden_size = n_hidden
        self.lstm = nn.LSTM(
            input_size = n_features,
            hidden_size = n_hidden,
            num_layers = n_layers,
            batch_first = True,
            dropout = 0.75
        )
        self.fc = nn.Linear(n_hidden, n_classes)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [15]:
model = SequenceModel(1,2,128,3)
model.to(device)

SequenceModel(
  (lstm): LSTM(1, 128, num_layers=3, batch_first=True, dropout=0.75)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

In [16]:
learning_rate = 0.0001
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0
    
    for batch_index, batch in enumerate(train_loader):
        sequence_batch , label_batch = batch['sequence'].to(device) , batch['label'].to(device) 
        output = model(sequence_batch)
        loss = loss_function(output, label_batch)
        running_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()

In [18]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0
    
    for batch_index, batch in enumerate(test_loader):
        sequence_batch , label_batch = batch['sequence'].to(device) , batch['label'].to(device) 
        
        with torch.no_grad():
            output = model(sequence_batch)
            loss = loss_function(output, label_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)
    
    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [19]:
num_epochs = 100
for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

Epoch: 1
Batch 100, Loss: 0.685
Batch 200, Loss: 0.337

Val Loss: 0.470
***************************************************

Epoch: 2
Batch 100, Loss: 0.137
Batch 200, Loss: 0.112

Val Loss: 0.225
***************************************************

Epoch: 3
Batch 100, Loss: 0.088
Batch 200, Loss: 0.081

Val Loss: 0.556
***************************************************

Epoch: 4
Batch 100, Loss: 0.084
Batch 200, Loss: 0.069

Val Loss: 0.106
***************************************************

Epoch: 5
Batch 100, Loss: 0.070
Batch 200, Loss: 0.064

Val Loss: 0.241
***************************************************

Epoch: 6
Batch 100, Loss: 0.058
Batch 200, Loss: 0.066

Val Loss: 0.399
***************************************************

Epoch: 7
Batch 100, Loss: 0.054
Batch 200, Loss: 0.056

Val Loss: 0.097
***************************************************

Epoch: 8
Batch 100, Loss: 0.053
Batch 200, Loss: 0.051

Val Loss: 0.078
***************************************************



In [20]:
with torch.no_grad():
    output = model(X_test.to(device))
    predicted = torch.argmax(output, dim=1)
    predicted ^= 1

In [44]:
arr = (predicted == torch.flatten(Y_test)).numpy()
final_test_acc = sum(arr)/len(arr)
final_test_acc

0.9949953660797034

In [45]:
torch.save(model, 'no_scaler_critic.pth')

In [46]:
# Model class must be defined somewhere
model_loaded = torch.load('no_scaler_critic.pth')
model_loaded.eval()

SequenceModel(
  (lstm): LSTM(1, 128, num_layers=3, batch_first=True, dropout=0.75)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)